1. Apriori algorithm:

    a. Find frequently occurring itemsets using the Apriori algorithm.

    b. Compute the support of the frequent itemset.

    c. Compute the confidence and lift of an association rule.

2. FP-Growth algorithm.

    a. Find frequently occurring itemsets using the FP-Growth
algorithm.

    b. Compute the support of the frequent itemset.

    c. Compute the confidence and lift of an association rule.

3. Compare Apriori and FP-growth algorithms.

In [9]:
!pip install mlxtend

In [10]:

from itertools import combinations

def load_data(fname):
    with open(fname) as f:
        return [line.strip().split(',')[1:] for line in f.readlines()[1:] if line.strip()]

def count_support(items, txs):
    return sum(all(i in t for i in items) for t in txs)

def apriori(txs, min_sup=0.1):
    N, minsup = len(txs), min_sup * len(txs)
    freqsets, k = [], 1
    items = sorted(set(i for t in txs for i in t))
    curr = [[i] for i in items if count_support([i], txs) >= minsup]
    freqsets += [(s, count_support(s, txs), count_support(s, txs)/N) for s in curr]

    while curr:
        cands = sorted(set(tuple(sorted(set(a + b))) for i, a in enumerate(curr) for b in curr[i+1:] if len(set(a + b)) == k+1))
        curr = [list(c) for c in cands if (c_s := count_support(list(c), txs)) >= minsup]
        freqsets += [(list(c), c_s, c_s/N) for c in curr]
        k += 1
    return freqsets

print("=== QUESTION 1a: APRIORI FREQUENT ITEMSETS ===\n")
for fname in ['sports.txt', 'space.txt']:
    print(f"File: {fname}\n" + "-"*30)
    try:
        txs = load_data(fname)
        res = apriori(txs)
        print(f"Total: {len(txs)}, Found: {len(res)}\n")
        for s, c, sup in res:
            print(f"{s}\n  Count: {c}, Support: {sup:.3f}")
    except FileNotFoundError:
        print(f"  File {fname} not found!")
    print("="*50 + "\n")


=== QUESTION 1a: APRIORI FREQUENT ITEMSETS ===

File: sports.txt
------------------------------
Total: 50, Found: 20

['cricket ball']
  Count: 18, Support: 0.360
['cricket bat']
  Count: 20, Support: 0.400
['football']
  Count: 22, Support: 0.440
['gloves']
  Count: 18, Support: 0.360
['ice cream']
  Count: 13, Support: 0.260
['juice']
  Count: 21, Support: 0.420
['water bottle']
  Count: 14, Support: 0.280
['cricket ball', 'cricket bat']
  Count: 4, Support: 0.080
['cricket ball', 'football']
  Count: 4, Support: 0.080
['cricket ball', 'gloves']
  Count: 4, Support: 0.080
['cricket ball', 'juice']
  Count: 4, Support: 0.080
['cricket bat', 'football']
  Count: 4, Support: 0.080
['cricket bat', 'gloves']
  Count: 4, Support: 0.080
['cricket bat', 'juice']
  Count: 4, Support: 0.080
['football', 'gloves']
  Count: 4, Support: 0.080
['football', 'ice cream']
  Count: 4, Support: 0.080
['football', 'juice']
  Count: 4, Support: 0.080
['football', 'water bottle']
  Count: 4, Support: 0.08

In [11]:

def load_data(f): return [l.strip().split(',')[1:] for l in open(f).readlines()[1:] if l.strip()]
def support(itemset, txs):
    c = sum(all(i in t for i in itemset) for t in txs)
    return c, c / len(txs) if txs else 0

def demo(txs):
    print("Support Examples:\n" + "-"*30)
    for s in [['Swimming'], ['Baseball', 'Football'], ['Tennis', 'Basketball', 'Baseball']]:
        c, sup = support(s, txs)
        print(f"{s} -> Count: {c}/{len(txs)}, Support: {sup:.3f}")

print("=== QUESTION 1b: SUPPORT CALCULATION ===\n")
for f in ['sports.txt', 'space.txt']:
    print(f"File: {f}\n" + "-"*30)
    try:
        txs = load_data(f)
        print(f"Total: {len(txs)}\n")
        demo(txs)
        print("Sample transactions:")
        for i, t in enumerate(txs[:3]):
            print(f"  T{i+1}: {t}")
    except FileNotFoundError:
        print(f"  File {f} not found!")
    print("="*50 + "\n")


=== QUESTION 1b: SUPPORT CALCULATION ===

File: sports.txt
------------------------------
Total: 50

Support Examples:
------------------------------
['Swimming'] -> Count: 0/50, Support: 0.000
['Baseball', 'Football'] -> Count: 0/50, Support: 0.000
['Tennis', 'Basketball', 'Baseball'] -> Count: 0/50, Support: 0.000
Sample transactions:
  T1: ['football', 'cricket ball', 'gloves']
  T2: ['cricket bat', 'cricket ball', 'juice']
  T3: ['football', 'water bottle', 'juice']

File: space.txt
------------------------------
Total: 50

Support Examples:
------------------------------
['Swimming'] -> Count: 0/50, Support: 0.000
['Baseball', 'Football'] -> Count: 0/50, Support: 0.000
['Tennis', 'Basketball', 'Baseball'] -> Count: 0/50, Support: 0.000
Sample transactions:
  T1: ['Robotic Arm', 'Food Packets']
  T2: ['Sleeping Bag', 'Treadmill', 'Food Packets']
  T3: ['Robotic Arm', 'Space Suit', '3D Printer']



In [12]:

from itertools import combinations

def load_data(f): return [l.strip().split(',')[1:] for l in open(f).readlines()[1:] if l.strip()]
def support(s, txs): return sum(all(i in t for i in s) for t in txs) / len(txs)

def find_freq_sets(txs, min_sup=0.1):
    items = set(i for t in txs for i in t)
    return [list(s) for k in [2, 3] for s in combinations(items, k) if support(s, txs) >= min_sup]

def assoc_rules(itemset, txs, min_conf=0.3):
    rules = []
    for i in range(1, len(itemset)):
        for A in combinations(itemset, i):
            B = [x for x in itemset if x not in A]
            sup_AB, sup_A, sup_B = support(itemset, txs), support(A, txs), support(B, txs)
            conf = sup_AB / sup_A if sup_A else 0
            lift = conf / sup_B if sup_B else 0
            if conf >= min_conf:
                rules.append({'antecedent': list(A), 'consequent': B, 'support': sup_AB,
                              'confidence': conf, 'lift': lift})
    return rules

print("=== QUESTION 1c: ASSOCIATION RULES (CONFIDENCE & LIFT) ===\n")
for fname in ['sports.txt', 'space.txt']:
    print(f"File: {fname}\n" + "-"*40)
    try:
        txs = load_data(fname)
        freq = find_freq_sets(txs)
        print(f"Transactions: {len(txs)}, Frequent sets: {len(freq)}\n")

        rules = [r for s in freq for r in assoc_rules(s, txs)]
        print("Association Rules:\n" + "-"*30)
        for i, r in enumerate(rules[:5], 1):
            print(f"Rule {i}: {r['antecedent']} → {r['consequent']}")
            print(f"  Support: {r['support']:.3f}, Confidence: {r['confidence']:.3f}, Lift: {r['lift']:.3f}")
            print("  →", "Positively associated" if r['lift'] > 1 else "Negatively associated" if r['lift'] < 1 else "Independent")
            print()
        if not rules: print("  No rules found with minimum confidence.")
    except FileNotFoundError:
        print(f"  File {fname} not found!")
    print("="*60 + "\n")


=== QUESTION 1c: ASSOCIATION RULES (CONFIDENCE & LIFT) ===

File: sports.txt
----------------------------------------
Transactions: 50, Frequent sets: 13

Association Rules:
------------------------------
Rule 1: ['cricket ball'] → ['juice']
  Support: 0.120, Confidence: 0.333, Lift: 0.794
  → Negatively associated

Rule 2: ['cricket ball'] → ['cricket bat']
  Support: 0.140, Confidence: 0.389, Lift: 0.972
  → Negatively associated

Rule 3: ['cricket bat'] → ['cricket ball']
  Support: 0.140, Confidence: 0.350, Lift: 0.972
  → Negatively associated

Rule 4: ['cricket ball'] → ['gloves']
  Support: 0.140, Confidence: 0.389, Lift: 1.080
  → Positively associated

Rule 5: ['gloves'] → ['cricket ball']
  Support: 0.140, Confidence: 0.389, Lift: 1.080
  → Positively associated


File: space.txt
----------------------------------------
Transactions: 50, Frequent sets: 7

Association Rules:
------------------------------
Rule 1: ['3D Printer'] → ['Sleeping Bag']
  Support: 0.100, Confidence: 

In [13]:

from collections import defaultdict, Counter

class FPNode:
    def __init__(s, i, c=0, p=None): s.item, s.count, s.parent, s.children = i, c, p, {}; s.next_node = None

ld = lambda f: [l.strip().split(',')[1:] for l in open(f).readlines()[1:] if l.strip()]

def insert(tx, n, h):
    if not tx: return
    i = tx[0]
    if i in n.children: n.children[i].count += 1
    else: n.children[i] = FPNode(i, 1, n); h[i].append(n.children[i])
    insert(tx[1:], n.children[i], h)

def build(txs, min_sup):
    N, r, h = len(txs), FPNode("root"), defaultdict(list)
    f = {i: c for i, c in Counter(i for t in txs for i in t).items() if c >= min_sup * N}
    for t in txs:
        ft = sorted([i for i in t if i in f], key=lambda x: -f[x])
        if ft: insert(ft, r, h)
    return r, h, f

def mine(h, min_sup, N, pre=[]):
    out = []
    for i, ns in sorted(h.items(), key=lambda x: sum(n.count for n in x[1])):
        path = pre + [i]
        total = sum(n.count for n in ns)
        if total >= min_sup * N:
            out.append((path, total, total/N))
            cond = []
            for n in ns:
                p, cur = [], n.parent
                while cur.item != "root": p.append(cur.item); cur = cur.parent
                cond += [p[::-1]] * n.count
            if cond:
                _, ch, _ = build(cond, min_sup)
                out += mine(ch, min_sup, len(cond), path)
    return out

fg = lambda txs, ms=0.1: mine(build(txs, ms)[1], ms, len(txs))

print("=== QUESTION 2a: FP-GROWTH FREQUENT ITEMSETS ===\n")
for f in ['sports.txt', 'space.txt']:
    print(f"File: {f}\n" + "-"*30)
    try:
        txs = ld(f)
        res = fg(txs)
        print(f"Transactions: {len(txs)}, Itemsets: {len(res)}\n")
        for s, c, sup in res:
            print(f"  {s}\n    Count: {c}, Support: {sup:.3f}")
    except FileNotFoundError:
        print(f"  File {f} not found!")
    print("="*50 + "\n")


=== QUESTION 2a: FP-GROWTH FREQUENT ITEMSETS ===

File: sports.txt
------------------------------
Transactions: 50, Itemsets: 49

  ['ice cream']
    Count: 13, Support: 0.260
  ['ice cream', 'gloves']
    Count: 2, Support: 0.154
  ['ice cream', 'gloves', 'cricket bat']
    Count: 1, Support: 0.500
  ['ice cream', 'cricket ball']
    Count: 2, Support: 0.154
  ['ice cream', 'cricket ball', 'juice']
    Count: 1, Support: 0.500
  ['ice cream', 'cricket ball', 'football']
    Count: 1, Support: 0.500
  ['ice cream', 'water bottle']
    Count: 3, Support: 0.231
  ['ice cream', 'water bottle', 'football']
    Count: 1, Support: 0.333
  ['ice cream', 'water bottle', 'juice']
    Count: 1, Support: 0.333
  ['ice cream', 'cricket bat']
    Count: 4, Support: 0.308
  ['ice cream', 'cricket bat', 'juice']
    Count: 1, Support: 0.250
  ['ice cream', 'cricket bat', 'football']
    Count: 1, Support: 0.250
  ['ice cream', 'football']
    Count: 5, Support: 0.385
  ['ice cream', 'juice']
    Coun

In [14]:

from collections import Counter

ld = lambda f: [l.strip().split(',')[1:] for l in open(f).readlines()[1:] if l.strip()]
sup = lambda iset, txs: (sum(all(i in t for i in iset) for t in txs), len(txs))

def show_support(txs):
    print("FP-Growth Support Calculation:\n" + "-"*40)
    cnt = Counter(i for t in txs for i in t)
    print("Top item frequencies:")
    for i, c in cnt.most_common(5):
        print(f"  {i}: {c} times, Support = {c/len(txs):.3f}")
    items = [i for i, _ in cnt.most_common(3)]
    for n in [2, 3]:
        if len(items) >= n:
            s, total = sup(items[:n], txs)
            print(f"  {items[:n]}: {s} times, Support = {s/total:.3f}")

def show_process(txs, min_s=0.1):
    print("\nFP-Growth Process:\n" + "-"*30)
    cnt = Counter(i for t in txs for i in t)
    min_c = min_s * len(txs)
    print(f"Minimum support count: {min_c:.1f}")
    freq = {i: c for i, c in cnt.items() if c >= min_c}
    print(f"Frequent items: {len(freq)}")
    for i, c in sorted(freq.items(), key=lambda x: -x[1]):
        print(f"  {i}: count={c}, support={c/len(txs):.3f}")
    print("\nSample ordered transactions (top 3):")
    for i, t in enumerate(txs[:3]):
        f = sorted([x for x in t if x in freq], key=lambda x: -freq[x])
        print(f"  Transaction {i+1}: {f}")

print("=== QUESTION 2b: FP-GROWTH SUPPORT CALCULATION ===\n")
for f in ['sports.txt', 'space.txt']:
    print(f"File: {f}\n" + "-"*40)
    try:
        txs = ld(f)
        print(f"Total transactions: {len(txs)}\n")
        show_support(txs)
        show_process(txs)
    except FileNotFoundError:
        print(f"  File {f} not found!")
    print("\n" + "="*60 + "\n")


=== QUESTION 2b: FP-GROWTH SUPPORT CALCULATION ===

File: sports.txt
----------------------------------------
Total transactions: 50

FP-Growth Support Calculation:
----------------------------------------
Top item frequencies:
  football: 22 times, Support = 0.440
  juice: 21 times, Support = 0.420
  cricket bat: 20 times, Support = 0.400
  cricket ball: 18 times, Support = 0.360
  gloves: 18 times, Support = 0.360
  ['football', 'juice']: 7 times, Support = 0.140
  ['football', 'juice', 'cricket bat']: 2 times, Support = 0.040

FP-Growth Process:
------------------------------
Minimum support count: 5.0
Frequent items: 7
  football: count=22, support=0.440
  juice: count=21, support=0.420
  cricket bat: count=20, support=0.400
  cricket ball: count=18, support=0.360
  gloves: count=18, support=0.360
  water bottle: count=14, support=0.280
  ice cream: count=13, support=0.260

Sample ordered transactions (top 3):
  Transaction 1: ['football', 'cricket ball', 'gloves']
  Transaction 2:

In [15]:
# Question 2c: Compute confidence and lift of association rules (FP-Growth)

from collections import defaultdict, Counter
from itertools import combinations

def load_data(filename):
    """Load transactions from file"""
    with open(filename, 'r') as f:
        lines = f.readlines()[1:]  # Skip header
        return [line.strip().split(',')[1:] for line in lines if line.strip()]

def calculate_support(itemset, transactions):
    """Calculate support for an itemset"""
    count = sum(1 for trans in transactions if all(item in trans for item in itemset))
    return count / len(transactions)

def simple_fpgrowth_frequent(transactions, min_support=0.1):
    """Simplified FP-Growth to find frequent itemsets"""
    # Count item frequencies
    item_counts = Counter()
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1

    min_count = len(transactions) * min_support
    frequent_items = [item for item, count in item_counts.items() if count >= min_count]

    frequent_itemsets = []

    # Add single items
    for item in frequent_items:
        support = calculate_support([item], transactions)
        frequent_itemsets.append(([item], support))

    # Add 2-itemsets
    for itemset in combinations(frequent_items, 2):
        support = calculate_support(list(itemset), transactions)
        if support >= min_support:
            frequent_itemsets.append((list(itemset), support))

    # Add 3-itemsets
    for itemset in combinations(frequent_items, 3):
        support = calculate_support(list(itemset), transactions)
        if support >= min_support:
            frequent_itemsets.append((list(itemset), support))

    return frequent_itemsets

def generate_association_rules_fpgrowth(frequent_itemsets, transactions, min_confidence=0.3):
    """Generate association rules from frequent itemsets found by FP-Growth"""
    rules = []

    # Only consider itemsets with 2+ items
    for itemset, itemset_support in frequent_itemsets:
        if len(itemset) < 2:
            continue

        # Generate all possible antecedent -> consequent combinations
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                consequent = [item for item in itemset if item not in antecedent]

                # Calculate support for antecedent
                antecedent_support = calculate_support(list(antecedent), transactions)
                consequent_support = calculate_support(consequent, transactions)

                if antecedent_support > 0 and consequent_support > 0:
                    # Confidence = support(itemset) / support(antecedent)
                    confidence = itemset_support / antecedent_support

                    # Lift = confidence / support(consequent)
                    lift = confidence / consequent_support

                    if confidence >= min_confidence:
                        rules.append({
                            'antecedent': list(antecedent),
                            'consequent': consequent,
                            'support': itemset_support,
                            'confidence': confidence,
                            'lift': lift
                        })

    return rules

def explain_metrics():
    """Explain what confidence and lift mean"""
    print("Association Rule Metrics Explanation:")
    print("-" * 40)
    print("• Support: How often the itemset appears")
    print("• Confidence: How often consequent appears when antecedent appears")
    print("• Lift: How much more likely consequent is when we have antecedent")
    print("  - Lift > 1: Positive association")
    print("  - Lift < 1: Negative association")
    print("  - Lift = 1: No association")
    print()

# Test with both files
print("=== QUESTION 2c: FP-GROWTH ASSOCIATION RULES ===\n")

for filename in ['sports.txt', 'space.txt']:
    print(f"File: {filename}")
    print("-" * 40)

    try:
        transactions = load_data(filename)
        print(f"Total transactions: {len(transactions)}\n")

        # Find frequent itemsets using simplified FP-Growth approach
        frequent_itemsets = simple_fpgrowth_frequent(transactions)
        print(f"Frequent itemsets found: {len(frequent_itemsets)}")

        # Show some frequent itemsets
        print("\nFrequent itemsets:")
        for itemset, support in frequent_itemsets[:5]:
            print(f"  {itemset} (support: {support:.3f})")

        # Generate association rules
        rules = generate_association_rules_fpgrowth(frequent_itemsets, transactions)

        print(f"\nAssociation Rules (min_confidence = 0.5):")
        print("-" * 50)

        if rules:
            explain_metrics()

            for i, rule in enumerate(rules[:5], 1):
                print(f"Rule {i}: {rule['antecedent']} → {rule['consequent']}")
                print(f"  Support: {rule['support']:.3f}")
                print(f"  Confidence: {rule['confidence']:.3f}")
                print(f"  Lift: {rule['lift']:.3f}")

                # Interpretation
                if rule['lift'] > 1.1:
                    print(f"  → Strong positive association!")
                elif rule['lift'] > 1:
                    print(f"  → Positive association")
                elif rule['lift'] < 0.9:
                    print(f"  → Negative association")
                else:
                    print(f"  → Weak/no association")
                print()
        else:
            print("  No rules found meeting minimum confidence threshold")

    except FileNotFoundError:
        print(f"  File {filename} not found!")

    print("\n" + "="*60 + "\n")

=== QUESTION 2c: FP-GROWTH ASSOCIATION RULES ===

File: sports.txt
----------------------------------------
Total transactions: 50

Frequent itemsets found: 20

Frequent itemsets:
  ['football'] (support: 0.440)
  ['cricket ball'] (support: 0.360)
  ['gloves'] (support: 0.360)
  ['cricket bat'] (support: 0.400)
  ['juice'] (support: 0.420)

Association Rules (min_confidence = 0.5):
--------------------------------------------------
Association Rule Metrics Explanation:
----------------------------------------
• Support: How often the itemset appears
• Confidence: How often consequent appears when antecedent appears
• Lift: How much more likely consequent is when we have antecedent
  - Lift > 1: Positive association
  - Lift < 1: Negative association
  - Lift = 1: No association

Rule 1: ['football'] → ['gloves']
  Support: 0.140
  Confidence: 0.318
  Lift: 0.884
  → Negative association

Rule 2: ['gloves'] → ['football']
  Support: 0.140
  Confidence: 0.389
  Lift: 0.884
  → Negative a